<code>years_em</code> fueron completados con ceros

<code>years_demency</code> fueron completados con ceros

<code>tecar</code> reasignado con binario

<code>lateralidad</code> reasignado con binario: 1:diestro

<code>genero</code> reasignado con binario: 1:M 0:H

<code>tipico</code>, <code>tipico</code> y <code>deposito</code> han sido completadas con ceros en las casillas vacías (Entre los existentes ninguno tiene valor cero)

Paciente 10: variable <code>posee_dii</code> vacío. Se ha puesto 'NO'

Paciente 25: variables <code>forst_iden</code>,	<code>fcsrt_free_recall_score</code>,	<code>fcsrt_cued_recall_score</code> y	<code>fcsrt_total_recall_score</code> vacías. Se ha puesto 0


Variables han sido reescritas en minúscula y separadas con _


In [2]:
# Imports needed for the script
from scipy import stats
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
import math
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [3]:
data_non = pd.read_csv("datos_new_selected_v4.csv", sep = ';') 

names = data_non.columns
#data_non.columns = [''] * len(data_non.columns)
data_non.shape

(70, 196)

In [4]:
data_non = data_non.fillna(0)

In [5]:
#data_non.index = data_non.index.droplevel(0)
#data_non = data_non.reset_index()
#data_non.set_index(data_non.columns[0]).reset_index()
data_non.head(70)

,nombre,edad,diagnostico,lado,lateralidad,genero,escolaridad,years_em,years_demency,posee_hta,...,wmrhfrontalpole_whitematter,wmrhtemporalpole_whitematter,wmrhtransversetemporal_whitematter,wmrhinsula_whitematter,leftunsegmentedwhitematter_whitematter,rightunsegmentedwhitematter_whitematter,lhcerebralwhitemattervol_whitematter,rhcerebralwhitemattervol_whitematter,cerebralwhitemattervol_whitematter,estimatedtotalintracranialvol_whitematter
0,esq-01-en,58,F203,1,DIESTRO,M,5,26,0,SI,...,237.5,624.9,596.2,8179.7,29409.6,29251.5,221752,222289,444042.0,1300000
1,esq-01-en,58,F203,2,DIESTRO,M,5,26,0,SI,...,237.5,624.9,596.2,8179.7,29409.6,29251.5,221752,222289,444042.0,1300000
2,esq-01-hc,62,C,1,DIESTRO,M,14,0,0,NO,...,234.1,643.2,680.8,9619.2,38461.8,37319.0,237701,238277,475978.0,1700000
3,esq-01-hc,62,C,2,DIESTRO,M,14,0,0,NO,...,234.1,643.2,680.8,9619.2,38461.8,37319.0,237701,238277,475978.0,1700000
4,esq-02-ed,79,F203-DEM.F001,1,DIESTRO,M,5,55,6,SI,...,475.5,1296.4,795.0,11020.7,21842.2,23099.9,264643,268662,533306.0,1200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,esq-13-hc,59,C,2,DIESTRO,M,16,0,0,NO,...,271.3,637.8,565.8,8341.8,27161.1,25354.3,206831,204116,410946.0,1300000
66,esq-14-hc,54,C,1,DIESTRO,M,18,0,0,NO,...,303.7,646.2,464.6,8241.7,26706.6,28135.7,208695,205077,413771.0,1200000
67,esq-14-hc,54,C,2,DIESTRO,M,18,0,0,NO,...,303.7,646.2,464.6,8241.7,26706.6,28135.7,208695,205077,413771.0,1200000
68,esq-15-hc,64,C,1,DIESTRO,M,16,0,0,NO,...,357.4,637.8,550.8,9450.3,27745.0,27229.7,228579,225270,453848.0,1400000


In [6]:
data_non['posee_hta_encoded'] = data_non['posee_hta'].map( {'SI': 1, 'NO': 0, 'si': 1, 'no': 0} ).astype(int)
data_non['tecar_encoded'] = data_non['tecar'].map( {'SI': 1, 'NO': 0, 'si': 1, 'no': 0} ).astype(int)
data_non['posee_dii_encoded'] = data_non['posee_dii'].map( {'SI': 1, 'NO': 0, 'si': 1, 'no': 0} ).astype(int)
data_non['genero_encoded'] = data_non['genero'].map( {'M': 1, 'H': 0, 'm': 1, 'h': 0} ).astype(int)
data_non['lateralidad_encoded'] = data_non['lateralidad'].map( {'DIESTRO': 1, 'ZURDO': 0, 'diestro': 1, 'zurdo': 0} )#.astype(int)
data_non['diagnostico_encoded'] = data_non['diagnostico'].map( {'C': 0, 
                                                        'F200': 1, 
                                                        'F203': 1, 
                                                        'F203-DEM.F000': 2,
                                                        'F203-DEM-Z864-FO01': 2,
                                                        'F203-DEM.F001': 2,
                                                        'F203-DEM.F011': 2,
                                                        'F203-DEM.F020': 2} )#.astype(int)



In [7]:
data_non=data_non.fillna(0)
data_non.head(67)

,nombre,edad,diagnostico,lado,lateralidad,genero,escolaridad,years_em,years_demency,posee_hta,...,lhcerebralwhitemattervol_whitematter,rhcerebralwhitemattervol_whitematter,cerebralwhitemattervol_whitematter,estimatedtotalintracranialvol_whitematter,posee_hta_encoded,tecar_encoded,posee_dii_encoded,genero_encoded,lateralidad_encoded,diagnostico_encoded
0,esq-01-en,58,F203,1,DIESTRO,M,5,26,0,SI,...,221752,222289,444042.0,1300000,1,0,0,1,1.0,1
1,esq-01-en,58,F203,2,DIESTRO,M,5,26,0,SI,...,221752,222289,444042.0,1300000,1,0,0,1,1.0,1
2,esq-01-hc,62,C,1,DIESTRO,M,14,0,0,NO,...,237701,238277,475978.0,1700000,0,0,0,1,1.0,0
3,esq-01-hc,62,C,2,DIESTRO,M,14,0,0,NO,...,237701,238277,475978.0,1700000,0,0,0,1,1.0,0
4,esq-02-ed,79,F203-DEM.F001,1,DIESTRO,M,5,55,6,SI,...,264643,268662,533306.0,1200000,1,1,0,1,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,esq-12-hc,57,C,1,DIESTRO,H,14,0,0,NO,...,264643,265777,530420.0,1500000,0,0,0,0,1.0,0
63,esq-12-hc,57,C,2,DIESTRO,H,14,0,0,NO,...,264643,265777,530420.0,1500000,0,0,0,0,1.0,0
64,esq-13-hc,59,C,1,DIESTRO,M,16,0,0,NO,...,206831,204116,410946.0,1300000,0,0,0,1,1.0,0
65,esq-13-hc,59,C,2,DIESTRO,M,16,0,0,NO,...,206831,204116,410946.0,1300000,0,0,0,1,1.0,0


In [70]:
drop_elements = ['nombre','posee_hta','posee_dii','genero','lateralidad','diagnostico','tecar','years_demency','tipico','atipico','deposito','meses_con_antipsicotico']
data_encoded = data_non.drop(drop_elements, axis = 1)
data_encoded.head(35)
names = data_encoded.columns

In [71]:
data_encoded.shape

(70, 190)

In [72]:
#data_encoded = data_encoded.replace('\d*\.?\d+','\d*\,?\d+', regex=True)
data_encoded=data_encoded[(data_encoded['lado']==2)]
data_encoded.shape

(35, 190)

In [10]:
tipo_grupo_todos_registros=[]
variables_correlacion_todos_registros=[]
valores_correlacion_todos_registros=[]

In [73]:
# Example
cor = data_encoded.corr()
idx_strs = []
r2_values = []


for i in range(1,24):
    for j in range(25,189):
        # Independent variables

        muestras = data_encoded[(data_encoded['diagnostico_encoded']==2)]

        #Demografia

        xs = muestras[['edad','escolaridad','years_em',muestras.columns[i]]]
        X2 = sm.add_constant(xs)

        #Estructura
        y = muestras[muestras.columns[j]]


        est = sm.OLS(y, X2)
        est2 = est.fit()
        if est2.rsquared > 0.7:
            print(est2.summary())
        #print("\n"+str(idx))
        #f.write(est2.summary().encode())
        #f.write("\n")


        #reg = LinearRegression().fit(xs, y)
        #reg.score(xs, y)
        #print(reg.intercept_)
        #idx = idx +1


                                  OLS Regression Results                                 
Dep. Variable:     parsopercularis_graymatter_xi   R-squared:                       0.828
Model:                                       OLS   Adj. R-squared:                  0.742
Method:                            Least Squares   F-statistic:                     9.612
Date:                           Sat, 11 Apr 2020   Prob (F-statistic):             0.0104
Time:                                   00:27:22   Log-Likelihood:                -52.499
No. Observations:                             10   AIC:                             113.0
Df Residuals:                                  6   BIC:                             114.2
Df Model:                                      3                                         
Covariance Type:                       nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------

                                  OLS Regression Results                                 
Dep. Variable:     parsopercularis_graymatter_xi   R-squared:                       0.828
Model:                                       OLS   Adj. R-squared:                  0.742
Method:                            Least Squares   F-statistic:                     9.612
Date:                           Sat, 11 Apr 2020   Prob (F-statistic):             0.0104
Time:                                   00:27:22   Log-Likelihood:                -52.499
No. Observations:                             10   AIC:                             113.0
Df Residuals:                                  6   BIC:                             114.2
Df Model:                                      3                                         
Covariance Type:                       nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------

                                  OLS Regression Results                                 
Dep. Variable:     parsopercularis_graymatter_xi   R-squared:                       0.828
Model:                                       OLS   Adj. R-squared:                  0.742
Method:                            Least Squares   F-statistic:                     9.612
Date:                           Sat, 11 Apr 2020   Prob (F-statistic):             0.0104
Time:                                   00:27:23   Log-Likelihood:                -52.499
No. Observations:                             10   AIC:                             113.0
Df Residuals:                                  6   BIC:                             114.2
Df Model:                                      3                                         
Covariance Type:                       nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------

                                  OLS Regression Results                                 
Dep. Variable:     parsopercularis_graymatter_xi   R-squared:                       0.834
Model:                                       OLS   Adj. R-squared:                  0.701
Method:                            Least Squares   F-statistic:                     6.283
Date:                           Sat, 11 Apr 2020   Prob (F-statistic):             0.0346
Time:                                   00:27:23   Log-Likelihood:                -52.312
No. Observations:                             10   AIC:                             114.6
Df Residuals:                                  5   BIC:                             116.1
Df Model:                                      4                                         
Covariance Type:                       nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------

                                OLS Regression Results                                
Dep. Variable:     wmlhentorhinal_whitematter   R-squared:                       0.794
Model:                                    OLS   Adj. R-squared:                  0.629
Method:                         Least Squares   F-statistic:                     4.818
Date:                        Sat, 11 Apr 2020   Prob (F-statistic):             0.0575
Time:                                00:27:23   Log-Likelihood:                -65.409
No. Observations:                          10   AIC:                             140.8
Df Residuals:                               5   BIC:                             142.3
Df Model:                                   4                                         
Covariance Type:                    nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------

...

                              OLS Regression Results                              
Dep. Variable:     thventricle_ventricles   R-squared:                       0.789
Model:                                OLS   Adj. R-squared:                  0.621
Method:                     Least Squares   F-statistic:                     4.687
Date:                    Sat, 11 Apr 2020   Prob (F-statistic):             0.0605
Time:                            00:27:35   Log-Likelihood:                -67.059
No. Observations:                      10   AIC:                             144.1
Df Residuals:                           5   BIC:                             145.6
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

                                    OLS Regression Results                                    
Dep. Variable:     wmrhtransversetemporal_whitematter   R-squared:                       0.763
Model:                                            OLS   Adj. R-squared:                  0.574
Method:                                 Least Squares   F-statistic:                     4.031
Date:                                Sat, 11 Apr 2020   Prob (F-statistic):             0.0793
Time:                                        00:27:35   Log-Likelihood:                -53.132
No. Observations:                                  10   AIC:                             116.3
Df Residuals:                                       5   BIC:                             117.8
Df Model:                                           4                                         
Covariance Type:                            nonrobust                                         
                              coef    std err     

                                  OLS Regression Results                                  
Dep. Variable:     lhhippocampal_tail_hippocampus   R-squared:                       0.840
Model:                                        OLS   Adj. R-squared:                  0.711
Method:                             Least Squares   F-statistic:                     6.541
Date:                            Sat, 11 Apr 2020   Prob (F-statistic):             0.0319
Time:                                    00:27:35   Log-Likelihood:                -48.947
No. Observations:                              10   AIC:                             107.9
Df Residuals:                                   5   BIC:                             109.4
Df Model:                                       4                                         
Covariance Type:                        nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975

                                OLS Regression Results                               
Dep. Variable:     wmlhprecuneus_whitematter   R-squared:                       0.824
Model:                                   OLS   Adj. R-squared:                  0.683
Method:                        Least Squares   F-statistic:                     5.856
Date:                       Sat, 11 Apr 2020   Prob (F-statistic):             0.0397
Time:                               00:27:35   Log-Likelihood:                -75.046
No. Observations:                         10   AIC:                             160.1
Df Residuals:                              5   BIC:                             161.6
Df Model:                                  4                                         
Covariance Type:                   nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

                             OLS Regression Results                            
Dep. Variable:     lateralidad_encoded   R-squared:                       0.745
Model:                             OLS   Adj. R-squared:                  0.541
Method:                  Least Squares   F-statistic:                     3.653
Date:                 Sat, 11 Apr 2020   Prob (F-statistic):             0.0940
Time:                         00:27:36   Log-Likelihood:                 1.8067
No. Observations:                   10   AIC:                             6.387
Df Residuals:                        5   BIC:                             7.900
Df Model:                            4                                         
Covariance Type:             nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const           

In [ ]:
# Example
cor = data_encoded.corr()
idx_strs = []
r2_values = []



for j in range(25,189):
    # Independent variables

     

    

    muestras = data_encoded[(data_encoded['diagnostico_encoded']==2)]
    
    #Demografia
    
    xs = muestras[['edad','escolaridad','years_em']]
    X2 = sm.add_constant(xs)
    
    #Estructura
    y = muestras[muestras.columns[j]]
    
    
    est = sm.OLS(y, X2)
    est2 = est.fit()
    if est2.rsquared > 0.6:
        print(est2.summary())
    #print("\n"+str(idx))
    #f.write(est2.summary().encode())
    #f.write("\n")

    
    #reg = LinearRegression().fit(xs, y)
    #reg.score(xs, y)
    #print(reg.intercept_)
    #idx = idx +1
